In [119]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

In [120]:
df = pd.read_csv("data/us_hotel_reviews.csv",encoding='ISO-8859-1')
df.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt lï¿½ï¿½ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½...,NaN,sungchul,NaN


In [121]:
df.dtypes

address                  object
categories               object
city                     object
country                  object
latitude                float64
longitude               float64
name                     object
postalCode               object
province                 object
reviews.date             object
reviews.dateAdded        object
reviews.doRecommend     float64
reviews.id              float64
reviews.rating          float64
reviews.text             object
reviews.title            object
reviews.userCity         object
reviews.username         object
reviews.userProvince     object
dtype: object

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35912 entries, 0 to 35911
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   address               35912 non-null  object 
 1   categories            35912 non-null  object 
 2   city                  35912 non-null  object 
 3   country               35912 non-null  object 
 4   latitude              35826 non-null  float64
 5   longitude             35826 non-null  float64
 6   name                  35912 non-null  object 
 7   postalCode            35857 non-null  object 
 8   province              35912 non-null  object 
 9   reviews.date          35653 non-null  object 
 10  reviews.dateAdded     35912 non-null  object 
 11  reviews.doRecommend   0 non-null      float64
 12  reviews.id            0 non-null      float64
 13  reviews.rating        35050 non-null  float64
 14  reviews.text          35890 non-null  object 
 15  reviews.title      

In [123]:
df.isnull().sum()

address                     0
categories                  0
city                        0
country                     0
latitude                   86
longitude                  86
name                        0
postalCode                 55
province                    0
reviews.date              259
reviews.dateAdded           0
reviews.doRecommend     35912
reviews.id              35912
reviews.rating            862
reviews.text               22
reviews.title            1622
reviews.userCity        19649
reviews.username           43
reviews.userProvince    18394
dtype: int64

My idea is to make an evaluation of the hotel rating (not the comments) of this dataset and once is clean and all the data clear I would like to compare this information with the updated one, by webscraping one or two of the most relevant agencies, search and metasearch sites (tripadvisor, trivago, booking.com, expedia, etc.), this second part will be done only with the city of San Diego. So, for now I will discard all the irrelevant information). 

In [124]:
df_clean = df.copy() # I make a copy.

# I create list comprehension of the columns I don't need:
column_names = df_clean.columns
columns_unuseful = [column_names[i] for i in [0,3,4,5,7,8,10,11,12,14,15,16,17,18]]
df_clean.drop(columns_unuseful, inplace=True, axis=1)

In [125]:
df_clean.head(5)

,categories,city,name,reviews.date,reviews.rating
0,Hotels,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Hotels,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Hotels,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Hotels,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Hotels,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [126]:
df_clean.isnull().sum()

categories          0
city                0
name                0
reviews.date      259
reviews.rating    862
dtype: int64

Since the most important variable is the reviews.rating, I need to discart all those null and 0 cases.

In [127]:
#Then I drop NaN values:
df_clean = df_clean.dropna()
df_clean.isna().sum()

categories        0
city              0
name              0
reviews.date      0
reviews.rating    0
dtype: int64

In [128]:
#And I keep only rating greater than 0 because those with rate 0 are not a real rate, but a null:
df_clean = df_clean[df_clean["reviews.rating"] >= 1]

In [129]:
df_clean.head(5)

,categories,city,name,reviews.date,reviews.rating
0,Hotels,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Hotels,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Hotels,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Hotels,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Hotels,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [130]:
#I clean the date format:
df_clean["reviews.date"]=pd.Series([i.split("T")[0] for i in df_clean["reviews.date"]]).astype(str)

In [131]:
df_clean.head(5)

,categories,city,name,reviews.date,reviews.rating
0,Hotels,Mableton,Hotel Russo Palace,2013-09-22,4.0
1,Hotels,Mableton,Hotel Russo Palace,2015-04-03,5.0
2,Hotels,Mableton,Hotel Russo Palace,2014-05-13,5.0
3,Hotels,Mableton,Hotel Russo Palace,2013-10-27,5.0
4,Hotels,Mableton,Hotel Russo Palace,2015-03-05,5.0


In [134]:
df_san_diego = df_clean[df_clean["city"] == "San Diego"]

In [136]:
df_san_diego.head(5)

,categories,city,name,reviews.date,reviews.rating
3750,Hotels,San Diego,Sheraton Mission Valley San Diego Hotel,2016-10-15,4.0
3751,Hotels,San Diego,Sheraton Mission Valley San Diego Hotel,2016-08-13,4.0
3752,Hotels,San Diego,Sheraton Mission Valley San Diego Hotel,2015-09-07,3.0
3753,Hotels,San Diego,Sheraton Mission Valley San Diego Hotel,2016-10-25,5.0
3754,Hotels,San Diego,Sheraton Mission Valley San Diego Hotel,2015-09-24,4.0


In [137]:
#I store the new df, to be used in a different jupyter notebook:
%store df_clean
%store df_san_diego

Stored 'df_clean' (DataFrame)
Stored 'df_san_diego' (DataFrame)


In [138]:
#Also I creat a new csv:
df_clean.to_csv(r'data\hotel_rates_us.csv', index = False)
df_san_diego.to_csv(r'data\hotel_rates_san_diego.csv', index = False)